# Prueba de selección DS optimizador

**Candidato:** Jorge Alexis Castillo Sepúlveda

**Fecha:** 10 de julio de 2023

## Preguntas de modelamiento y programación

### Pregunta 1: Maximizando la utilidad de una línea aérea

El problema planteado tiene las características para ser modelado como un problema de programación lineal. La función a optimizar es una función de utilidad, que se define como las ganancias menos los costos:

$U=\text{ganancias}-\text{costos}$

En donde las ganancias están dadas por los ingresos debido a la venta de asientos más los beneficios por cada azafata y auxiliar adicional sobre el mínimo, mientras que los costos están dados por el pago de salarios, la comida servida a los pasajeros y el mantenimiento. Es claro que las variables de la función de utilidad son el número de asientos vendios por cada clase, el número de azafatas y el número de auxiliares. Esta función de utilidad está restringida por la capacidad de asientos por clase, de personal, de espacio físico y la relación que debe existir entre personal y número de asientos de manera de garantizar un buen servicio. En lo que sigue, se detallan cada una de estas componentes.

#### Variables

Sean $x_1$, $x_2$ y $x_3$ el número de asientos vendidos en la clase primera, ejecutiva y económica, respectivamente, y sean $y_1$ e $y_2$ el número de azafatas y auxiliares.   
En base a estas variables, podemos ir escribiendo las formulaciones de las componentes de la función de utilidad y las restricciones.


#### Ganancias

- Asientos: $2000x_1+1300x_2+900x_3$
- Beneficios por cada azafata y auxiliar adicional sobre el mínimo: $100(y_1-8)+50(y_2-2)$. Se hace una traslación respecto a 8 y 2 en el numero de azafatas y auxiliars resp., ya que esos vaores son los mínimos exigido.

#### Costos
- Salarios: $200y_1+120y_2$
- Comida: $80x_1+60x_2+50x_3$
- Mantenimiento: $75000$ (es fijo y constante)

#### Función objetivo

Remplazando cada una de estas componentes en la expresión de la utilidad arriba, se tiene que la función a maximizar es

$$U = 2000x_1+1300x_2+900x_3+100(y_1-8)+50(y_2-2)-(200y_1+120y_2+80x_1+60x_2+50x_3+75000)$$

#### Restricciones

- Capacidad de asientos para la clase primera: $25\leq x_1 \leq 45$
- Capacidad de asientos para la clase ejecutiva: $80\leq x_2 \leq 100$
- Capacidad de asientos para la clase económica: $120\leq x_3\leq 210$
- Número de azafatas: $8\leq y_1\leq 18$
- Número de auxiliares: $2\leq y_2 \leq 5$ 
- Espacio físico: $1.8\cdot x_1+1.4\cdot x_2+1\cdot x_3\leq 420$
- Razón entre asientos y azafatas para buen servicio: $\dfrac{x_1}{10}\leq y_1$,$\dfrac{x_2}{20}\leq y_1$, $\dfrac{x_3}{40}\leq y_1$ 
- Razón entre asientos y auxiliares para buen servicio: $\dfrac{x_1+x_2+x_3}{100}\leq y_2$

Además, es deseable que las variables $x_i$, $y_j$, con $i\in \{1,2,3\}$, $j\in \{1,2\}$ deben ser números enteros. 


#### Implementación 

Se usa la librería Pulp (ref: https://coin-or.github.io/pulp/index.html) para resolver este problema. Se debe tener especial cuidado con las restricciones que refieren a la razón entre asientos y personal, ya que la librería no soporta la operación de división entre variables de Pulp y números enteros. Para ello, se debe replantear el problema, eliminando las divisiones paa mantener la "linealidad" del problema. Por ejemplo, $\dfrac{x_1}{10}\leq y_1$ se debe rescribir como $x_1-10y_1\leq  0$. Otro punto relevante es que el rango donde se definen de las variables (lowBound, upBound) se puede escribir en la definición de estas mismas y no como restricciones.

El código que lo resuelve se muestra a continuación:


In [1]:
from pulp import *

# Definición del problema
prob = LpProblem("Maximizando la utilidad de una línea aérea", LpMaximize)

# Definición de variables
x1 = LpVariable("Primera", 25, 45,LpInteger) # el segundo argumento indica que la variable debe ser un número entero 
x2 = LpVariable("Ejecutiva", 80, 100,LpInteger)
x3 = LpVariable("Economica", 120, 210,LpInteger)
y1 = LpVariable("Azafatas", 8, 18,LpInteger)
y2 = LpVariable("Auxiliares", 2, 5,LpInteger)

# Función objetivo
prob += 2000*x1 + 1300*x2 + 900*x3+ (100*(y1-8) + 50*(y2-2)) - (200*y1 + 120*y2 + 80*x1 + 60*x2 + 50*x3 + 75000), "U"

# Restricciones
prob += 1.8*x1 + 1.4*x2 + x3 <= 420, "Restricción_espacio_físico"
prob += x1 - 10*y1 <= 0, "Razón entre azafatas y asientos de primera clase"
prob += x2 - 20*y1 <= 0, "Razón entre azafatas y asientos de clase ejecutiva"
prob += x3 - 40*y1 <= 0, "Razón entre azafatas y asientos de clase económica"
prob += x1 + x2 + x3 - 100*y2 <= 0, "Razón entre auxiliares y total de asientos"

# Resolver el problema
prob.solve()

# Imprimir el resultado
print("Estado:", LpStatus[prob.status])
for v in prob.variables():
    print(v.name, "=", v.varValue)
print("Utilidad máxima = ", value(prob.objective))


Estado: Optimal
Auxiliares = 4.0
Azafatas = 8.0
Economica = 199.0
Ejecutiva = 100.0
Primera = 45.0
Utilidad máxima =  302570.0


C:\Users\aleph\anaconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Los resultados muestran que para maximizar la Utilidad de la línea aérea en el proceso de incorporación del nuevo avión, se deben contratar 4 auxiliars y 8 azafatas, y elavión debe tener 199 asientos de clase económica, 100 asientos de clase ejecutiva y 45 asientos de primera clase. La utilidad generada será de 302,570 dólares. Los resultados parecen ser intuitivos de acuerdo a lo que se conoce del negocio de las aerolíneas y características físicas de un avión. 

### Problema 2: Puzzle de alimentos

Se tienen 5 comidas diferentes (ensalada, sopa,carne, pescado y pasta) que se eben distribuir en 5 días de acuerdo a ciertos requerimientos nutricionales que se dividen en 3 categorías (rojo, naranjo y verde). Mi interpretación de los dibujos que se muestran en la prueba es como sigue:

##### Apotes nutricionales

- **ensalada:** 2 naranjos + 1 rojo
- **sopa:** 1 naranjo + 2 rojos + 1 verde
- **carne:** 1 naranjos + 3 rojos + 3 verdes
- **pasta:** 3 naranjos + 1 rojo + 2 verde
- **pescado:**  1 naranjo + 3 rojos + 2 verdes

##### Requerimientos nutricionales por día

- **dia 1**: 8 naranjos+10 rojos+8 verde
- **dia 2:** 7 naranjos+9 rojos+5 verdes
- **dia 3:** 8 naranjos+10 rojos+9 verdes
- **dia 4:** 8 naranjos+11 rojos+9 verdes
- **dia 5:** 9 naranjos+10 rojos+10 verdes

**Disclaimer:** Me costó identificar/diferenciar la carne y la pasta, y asumí que son las que se indican arriba ya que parece ser que el nutriente naranjo son carbohidratos y el nutriente rojo son proteínas. 


Sean $x_{i,j}$ la cantidad de platos de la comida $i$ en el día $j$, en donde las comidas $i$ están ordenadas bajo el mismo orden que se mencionan en el enunciado (ensalada, sopa,carne, pescado y pasta) y por simplicidad acá se enumeran del 1 al 5 haciendo una correspondencia biunívoca entre las comidas odenadas de esa forma y $\{1,\ldots,5\}$. Bajo los supuestos hechos arriba, se tienen que las restricciones están dadas por 

**Nutriente naranja**

- Dia 1: $2x_{1,1}+x_{2,1}+x_{3,1}+3x_{4,1}+x_{5,1}=8$
- Día 2: $2x_{1,2}+x_{2,2}+x_{3,2}+3x_{4,2}+x_{5,2}=7$ 
- Día 3: $2x_{1,3}+x_{2,3}+x_{3,3}+3x_{4,3}+x_{5,3}=8$
- Día 4: $2x_{1,4}+x_{2,4}+x_{3,4}+3x_{4,4}+x_{5,4}=8$
- Día 5: $2x_{1,5}+x_{2,5}+x_{3,5}+3x_{4,5}+x_{5,5}=9$

**Nutriente rojo**

- Día 1: $x_{1,1}+2x_{2,1}+3x_{3,1}+x_{4,1}+3x_{5,1}=10$
- Día 2: $x_{1,2}+2x_{2,2}+3x_{3,2}+x_{4,2}+3x_{5,2}=9$
- Día 3: $x_{1,3}+2x_{2,3}+3x_{3,3}+x_{4,3}+3x_{5,3}=10$
- Día 4: $x_{1,4}+2x_{2,4}+3x_{3,4}+x_{4,4}+3x_{5,4}=11$
- Día 5: $x_{1,5}+2x_{2,5}+3x_{3,5}+x_{4,5}+3x_{5,5}=10$

**Nutriente verde**

- Día 1: $x_{2,1}+3x_{3,1}+2x_{4,1}+2x_{5,1}=8$
- Día 2: $x_{2,2}+3x_{3,2}+2x_{4,2}+2x_{5,2}=5$
- Día 3: $x_{2,3}+3x_{3,3}+2x_{4,3}+2x_{5,3}=9$
- Día 4: $x_{2,4}+3x_{3,4}+2x_{4,4}+2x_{5,4}=9$
- Día 5: $x_{2,5}+3x_{3,5}+2x_{4,5}+2x_{5,5}=10$

**Máximo de 5 platos por comida**

- $\forall j \in\{1,\ldots,5\}: \sum_i x_{i,j}\leq 5$

Se han definido las restricciones del problema, pero, **¿cuál es la función objetivo?** Ingenuamente se podría pensar que hay que minimizar cumplir *al menos* el requerimiento nutricional, pero este supuesto no sería correcto, ya que no es deseable, en términos de un contexto "nutricional", consumir más de lo que se requiere. De la misma manera se podría pensar en la otra dirección: no es deseable tener menos nutrientes que los requeridos. En el contexto del desafío en sí, se tiene que resolver un puzzle-acertijo tipo juego de casino para pdoer avanzar en las etapas de un juego más general, y el requerimiento es que no hay ni que minimizar ni maximizar ninguna cantidad en particular. Sólo se deben encontrar los valores $x_{i,j}$ que satisfagan las condiciones de los requerimientos. Para hacer esto en Pulp, se define la función objetivo simplemente como "0". A continuación, se deja la implementación en Python usando Pulp, en donde el output tendrá las cantidades de cada comida por día buscadas para avanzar en el juego: 

In [2]:
from pulp import *
# Variables
comidas = ['ensalada', 'sopa', 'carne', 'pescado', 'pasta']
dias = range(1, 6)

# Parámetros
aportes = {'ensalada': [2, 1, 0], 'sopa': [1, 2, 1], 'carne': [1, 3, 3], 
           'pescado': [1, 3, 2], 'pasta': [3, 1, 2]}  # [naranjos, rojos, verdes]

requerimientos = {1: [8, 10, 8], 2: [7, 9, 5], 3: [8, 10, 9], 
                  4: [8, 11, 9], 5: [9, 10, 10]}  # [naranjos, rojos, verdes]

# Crear la variable de problema
prob = LpProblem("Plan_de_comidas", LpMinimize)

# Crear las variables de decisión
x = LpVariable.dicts("cantidad", [(i,j) for i in comidas for j in dias], 0, 5, LpInteger)

# Función objetivo
prob += 0

# Restricciones de los requerimientos diarios
for j in dias:
    for k, nutriente in enumerate(["naranjo", "rojo", "verde"]):
        prob += lpSum(x[(i,j)] * aportes[i][k] for i in comidas) == requerimientos[j][k], f"Requerimiento_{nutriente}_{j}"

# Restricciones de la cantidad máxima de platos por día
for j in dias:
    prob += lpSum(x[(i,j)] for i in comidas) <= 5, f"MaxPlatos_{j}"

# Resolviendo el problema
prob.solve()

# Imprimir el estado y la solución del problema
print("Estado:", LpStatus[prob.status])
for v in prob.variables():
    print(v.name, "=", v.varValue)


Estado: Optimal
__dummy = None
cantidad_('carne',_1) = 1.0
cantidad_('carne',_2) = 1.0
cantidad_('carne',_3) = 2.0
cantidad_('carne',_4) = 1.0
cantidad_('carne',_5) = 1.0
cantidad_('ensalada',_1) = 1.0
cantidad_('ensalada',_2) = 2.0
cantidad_('ensalada',_3) = 1.0
cantidad_('ensalada',_4) = 1.0
cantidad_('ensalada',_5) = 0.0
cantidad_('pasta',_1) = 1.0
cantidad_('pasta',_2) = 0.0
cantidad_('pasta',_3) = 1.0
cantidad_('pasta',_4) = 1.0
cantidad_('pasta',_5) = 2.0
cantidad_('pescado',_1) = 1.0
cantidad_('pescado',_2) = 0.0
cantidad_('pescado',_3) = 0.0
cantidad_('pescado',_4) = 2.0
cantidad_('pescado',_5) = 1.0
cantidad_('sopa',_1) = 1.0
cantidad_('sopa',_2) = 2.0
cantidad_('sopa',_3) = 1.0
cantidad_('sopa',_4) = 0.0
cantidad_('sopa',_5) = 1.0


Se ha logrado llegar a una solución factible a este problema de programación entera y por lo tanto, la aventura continúa! 